In [1]:
import numpy as np
import copy 
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_probability as tfp
# Main C3 objects
from c3.c3objs import Quantity as Qty
from c3.parametermap import ParameterMap as PMap
from c3.experiment import Experiment as Exp
from c3.model import Model as Mdl
from c3.generator.generator import Generator as Gnr

# Building blocks
import c3.generator.devices as devices
import c3.signal.gates as gates
import c3.libraries.chip as chip
import c3.signal.pulse as pulse
import c3.libraries.tasks as tasks

# Libs and helpers
import c3.libraries.algorithms as algorithms
import c3.libraries.hamiltonians as hamiltonians
import c3.libraries.fidelities as fidelities
import c3.libraries.envelopes as envelopes
import c3.utils.qt_utils as qt_utils
import c3.utils.tf_utils as tf_utils

# Plotting
import plotly.express as px
import plotly.graph_objects as go
from plotting import plot_dynamics_plotly

# Utilities functions
import utilities_functions as ut

In [2]:
Num_qubits = 2
Num_coupler = Num_qubits - 1

qubit_levels = [3, 3]
coupler_levels = [3]

freq_q_arr = [4e9, 4e9]
anharm_q_arr = [-210e6, -210e6]
t1_q_arr = [27e-6, 27e-6]
t2star_q_arr = [39e-6, 39e-6]
phi_q = [0.1, 0.1]
phi_0_q = [10.0, 10.0]
d_q = [0.0, 0.0]

freq_c_arr = [4e9]
anharm_c_arr = [-210e6]
t1_c_arr = [27e-6]
t2star_c_arr = [39e-6]
phi_c = [0.1]
phi_0_c = [10.0]
d_c = [0.0]

qubit_temp = 50e-3


In [10]:
qubits = ut.intialize_qubits(
                    Num_qubits=Num_qubits,
                    qubit_levels_list=qubit_levels,
                    freq_list=freq_q_arr,
                    anharm_list=anharm_q_arr,
                    t1_list=t1_q_arr,
                    t2star_list=t2star_q_arr,
                    phi_list=phi_q,
                    phi0_list=phi_0_q,
                    d_list=d_q,
                    qubit_temp=qubit_temp
)

couplers = ut.initialize_coupler(
                    Num_coupler=Num_coupler,
                    coupler_levels_list=coupler_levels,
                    freq_list=freq_c_arr,
                    anharm_list=anharm_c_arr,
                    t1_list=t1_c_arr,
                    t2star_list=t2star_q_arr,
                    phi_list=phi_c,
                    phi0_list=phi_0_c,
                    d_list=d_c,
                    qubit_temp=qubit_temp
)

for i in qubits:
    print(i.name)

for i in couplers:
    print(i.name)

Q1
Q2
C1


In [12]:
NN_coupling_strength = [20e6, 20e6]
NNN_coupling_strength = []

couplings = ut.generate_couplings(
                Num_qubits=Num_qubits,
                Num_coupler=Num_coupler,
                NN_coupling_strength=NN_coupling_strength,
                NNN_coupling_strength=NNN_coupling_strength,
                qubit_array=qubits,
                coupler_array=couplers
)

for i in couplings:
    print(i.name, i.connected)

Q1-C1 ['Q1', 'C1']
Q2-C1 ['Q2', 'C1']


In [11]:
drives = ut.initialize_qubit_drives(
                Num_qubits=Num_qubits,
                qubit_array=qubits
)

for i in drives:
    print(i.name, i.connected)

d1 ['Q1']
d2 ['Q2']


In [13]:
init_temp = 50e-3
init_ground = tasks.InitialiseGround(
        init_temp = Qty(
                value = init_temp,
                min_val = -0.001,
                max_val = 0.22,
                unit = "K"
        )
)

In [14]:
model = Mdl(
        qubits + couplers,
        drives + couplings
)
model.set_lindbladian(False)
model.set_dressed(True)

C:\Users\Ashutosh\AppData\Local\Programs\Python\Python39\lib\site-packages\c3\model.py:384: UserWarning: C3 Warning: Some states are overly dressed, trying to recover...18 states, 2 is lowest failed state
  warnings.warn(


In [15]:
sim_res = 100e9
awg_res = 2e9
v2hz = 1e9

generator = ut.build_generator(
                Num_qubits=Num_qubits,
                drive_array=drives,
                sim_res=sim_res,
                awg_res=awg_res,
                v2hz=v2hz
)